<br>**GENETIC ALGORITHM**<font color='violet'></br>
<br>Genetic algorithm is an approach to find the fittest solution of problems inspired by Charles Darwin's theory of natural evolution. The algorithm uses the process of natural selection where the individuals as members of a population, which are usualy of great size, who best satisfy the given conditions are most likely to be chosen to generate the next generation. Hence the next generation is born with more satisfactory characteristics than the previous one and this process continues untill an individual is born which satisfies all conditions given or is best suited to fit a set of data. 
Five concepts needs to be defined in this approach:</font></br><font color='blue'>
1. initial population : It's a set of individuals which are normally chosen by random and hence they are most likely to be different in characteristics. In this problem population members are circuits which consist of a, to some extent, unique combination of logic gates.
    
2. chromosome (DNA):Each individual in a given population. Each DNA is a combination of the logic gates given.

3. Genes : Each DNA/chromosome cosists of characteristics which have combined together, in different types or different sequences, to form a DNA. In the given problem each gene could be one of the logic gates from the list {'AND','OR','NAND','NOR','XOR','XNOR'}.

4. selection : As in the Darwinian theory of slelection, individuals with more favorable characteristics are more likely to survive. Hence, we should give the DNAs wich better suit the data a higher chance to be chosen to produce the next generation. On the other hand, we shall not neglect other individuals as they might also have favorable characteristics which may contribute to the evolution of a more satisfying generation. The approach to select the parents to mate is to be explained:
        1. We choose a random member of the present population 
        2. We choose a random number in (=0,sum of fitness of the whole population)
        3. If the fitness value of the chosen member is less than the fitness value which we picked by chance, the member is validated to be a parent. This way we give members a chance of being selected proportional to their fitness value.

5. crossover: When two parents are selected by the previous approach, we choose a random number from (=0,length of DNA) as the midpoint and create two instances of new DNAs as children to which genes of first parents are assigned to before (first child) and after (second child) the midpoint.

6. mutation: after each child has been created, each child's genes begin to change randomly regarding to a mutation possibility given. This possibility should be low as we don't want our child deviate from their parents' favorable characteristics that much.

7. fitness : Fitness for each DNA is defined as the 2 to the power of the number of data sets the DNA can perfectly match given the inputs and the desired output (this number is named as DNA's score). With this choice of fitness function we are able to exaggerate the distinction between a regular DNA and a more favorable one.
</font>
And done. The population continues to evolve and change do to the reproduction. 
Further explanation of the code is to be given in the next section.

In [8]:
import numpy as np
import pandas as pd
from collections import deque
import datetime
import secrets
import csv 

filename='truth_table.csv'
FALSE='FALSE'
TRUE='TRUE'
NUMBER_OF_GATES=6 #TYPES
NUMBER_OF_INS=10

#indices in the "gates" list 
AND=0
OR=1
XOR=2
NAND=3
NOR=4
XNOR=5

def AND_GATE(in1,in2):
    return in1 & in2 

def OR_GATE(in1,in2):
    return in1 | in2

def NAND_GATE(in1,in2):
    return not (in1 & in2)

def NOR_GATE(in1,in2):
    return not (in1 | in2)

def XOR_GATE(in1,in2):
    return in1 ^ in2

def XNOR_GATE(in1,in2):
    return not (in1 ^ in2)

gates=[AND_GATE,OR_GATE,NAND_GATE,NOR_GATE,XOR_GATE,XNOR_GATE]
gates_names=['AND','OR','NAND','NOR','XOR','XNOR']

def import_data(filename):
    truth_table=[]
    counter=0
    with open(filename,newline='') as csvfile:
        sreader=csv.reader(csvfile,delimiter=',')
        for row in sreader:
            counter=counter+1
            if counter>1: #ignore the header line
                truth_table.append([(0 if x==FALSE else 1) for x in row])
    return truth_table

In [28]:
#generating a population of size n=1024 considering the worst case
#in which each of the random members population have the chance to 
#give the correct answer to at least one data set

class DNA:
    def __init__(self,DNA_length,ID):
        self.genes=[]
        self.score=0 #num of the lines which perfectly suits the DNA
        self.fitness=0
        self.ID=ID
        self.initialize_DNA_randomly(DNA_length)

    def initialize_DNA_randomly(self,DNA_length):
        for i in range(DNA_length):
            self.genes.append(secrets.randbelow(NUMBER_OF_GATES))

    def crossover(self,partner):
        child1=DNA(len(self.genes),self.ID)
        child2=DNA(len(self.genes),partner.ID)
        midpoint=secrets.randbelow(len(self.genes))
        for i in range(len(self.genes)):
            if i>=midpoint:
                child1.genes[i]=self.genes[i]
                child2.genes[i]=partner.genes[i]
            else:
                child1.genes[i]=partner.genes[i]
                child2.genes[i]=self.genes[i]

        child1.new_born=1
        child2.new_born=1
        
        return child1,child2
    
    def mutate(self,mutation_rate):
        for i in range(len(self.genes)):
            if(secrets.randbelow(NUMBER_OF_INS)/NUMBER_OF_INS<mutation_rate):
                self.genes[i]=secrets.randbelow(NUMBER_OF_GATES)

def evaluate_per_set(truth_set,DNA):
    temp=truth_set[0]
    for i in range(len(DNA.genes)):
        gate=gates[DNA.genes[i]]
        temp=gate(temp,truth_set[i+1])
    if temp==truth_set[-1]:
        DNA.score=DNA.score+1
    DNA.fitness=2**DNA.score 
    #an exponential fitness exaggerates the distinction
    #between DNAs with different scores which gives the 
    #DNAs to a higher chance to be chosen as a parent 
    #to generate the next population 
        
def evaluate(truth_table,population):
    for member in population:
        for truth_set in truth_table:
            evaluate_per_set(truth_set,member)
        if member.fitness==2**len(truth_table):
            return member
    return None
            
def setup_environment(population_size,DNA_length):
    population=[]
    for i in range(population_size):
        new_DNA=DNA(DNA_length,i)
        population.append(new_DNA)
    return population

def reproduct(population):
    eval_sum=0  
    for member in population:
        eval_sum=eval_sum+member.fitness
    mating_happened=0   
    parent1=choose_parent(population,eval_sum)
    parent2=choose_parent(population,eval_sum)
    child1,child2=parent1.crossover(parent2)
    population[parent1.ID]=child1
    population[parent2.ID]=child2
    del parent1
    del parent2
    child1.mutate(0.2)
    child2.mutate(0.2)
        
def choose_parent(population,eval_sum):
    count=0
    while True: 
        r1=secrets.randbelow(len(population))
        r2=secrets.randbelow(eval_sum)
        if population[r1].fitness<r2:
            return population[r1]
        count=count+1
        if count>10000: #avoiding infinite loops just in case
            print("stop")
            break

            
def GA(file_name):
    valid_answer=None
    truth_table=import_data(file_name)
    population=setup_environment(len(truth_table),len(truth_table[0])-2)
    count=0
    while not valid_answer:
        target=evaluate(truth_table,population)
        reproduct(population)
        count=count+1
        if target:
            valid_answer=1
    show_answer(target.genes)
    print(target.genes)
    #print(target.score)
    #print(target.fitness)
    #print(2**1024)

def show_answer(target):
    for gate in target:
        print(gates_names[gate],end=" ")
        print("\t",end=" ")
GA(filename)

XOR 	 NAND 	 NAND 	 XNOR 	 NAND 	 NAND 	 OR 	 XNOR 	 AND 	 [4, 2, 2, 5, 2, 2, 1, 5, 0]


In [155]:
NAND_GATE(XNOR_GATE(AND_GATE(XNOR_GATE(AND_GATE(AND_GATE(XNOR_GATE(NAND_GATE(OR_GATE(0,0),0),0),0),0),0),0),0),0)

True

In [24]:
score=0
def evaluate_set(truth_set,genes):
    temp=truth_set[0]
    score=0
    for i in range(len(genes)):
        gate=gates[genes[i]]
        temp=gate(temp,truth_set[i+1])
    if temp==truth_set[-1]:
        score=score+1     
    return score
        
def evaluate_single(genes):
    score=0
    truth_table=import_data(filename)
    for truth_set in truth_table:
        score=score+evaluate_set(truth_set,genes)
    if score==len(truth_table):
        print("irght")
        return
    print(score)

In [29]:
!jupyter nbconvert --to html AI_CA2_810098018.ipynb

[NbConvertApp] Converting notebook AI_CA2_810098018.ipynb to html
[NbConvertApp] Writing 307159 bytes to AI_CA2_810098018.html
